<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#local-mysql에-적재하는-코드-짜놓기" data-toc-modified-id="local-mysql에-적재하는-코드-짜놓기-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>local mysql에 적재하는 코드 짜놓기</a></span></li></ul></div>

- 출처 : 축산물품질평가원 공공 데이터 포탈 API이용

In [57]:
# 한 번에 가져오는 건 일일 트래픽 1000을 초과하기 때문에 2년치 씩만 가져와서 DB에 적재하자..
# 오늘 밤 12시 지나서 하기

    abattcode  작업장 코드
    abattnm  도매시장명
    auctDate  경매일자
    hanwooauctamt  한우 평균 경매 금액
    hanwooauct_0bamt 한우 1B+ 경매단가
    hanwooauct_4bamt 한우 3B 경매단가
    hanwooauctdiffamt  한우 평균 경매 금액 전날 차이
    hanwooauctexpectcnt 한우 경매 낙찰 두수
    skinyauctamt 돼지 탕박 경매 금액
    skinyautdiffamt 돼지 탕박 전날 경매금액 차이
    skinyauctcnt 돼지 탕박 경매 두수
    totalpigauctcnt 돼지 경매 두수
    yukwooauctamt 육우 경매 금액
    yukwooauctdiffamt 육우 전날 경매 금액 차이
    yukwooauctcnt 육우 경매 두수

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [ ]:
# 축산물 실시간 경매시황중계정보 - [전국] 부분만 가져오자!
# auctDate파라미터에 날짜를 loop문 주어야 함!
# auctFlag = 1(전체)로 고정 설정
datetimes = pd.date_range(start='20130107', end='20150107', freq='D').strftime('%Y%m%d').tolist()

data_dict = {}
# 변수별로 데이터 담을 리스트 정의
stores = []
dates = []
hanwoo_amt = []
hanwoo_0b_amt = []
hanwoo_4b_amt = []
pork_amt = []
yukwoo_amt = []

for datetime in datetimes:
    url = f'http://data.ekape.or.kr/openapi-data/service/user/grade/liveauct?serviceKey=0OhBU7ZCGIobDVKDeBJDpmDRqK3IRNF6jlf%2FJB2diFAf%2FfR2czYO9A4UTGcsOwppV6W2HVUeho%2FFPwXoL6DwqA%3D%3D&auctDate={datetime}&auctFlag=2'
    res = requests.get(url)
    soup = BeautifulSoup(res.content, 'html.parser')
    print(datetime)
    
    # 변수별 값 다 담기
    stores.append(soup.select_one('items > item:nth-of-type(17) > abattnm').get_text())
    dates.append(soup.select_one('items > item:nth-of-type(17) > auctdate').get_text())
    hanwoo_amt.append(soup.select_one('items > item:nth-of-type(17) > hanwooauctamt').get_text())
    hanwoo_0b_amt.append(soup.select_one('items > item:nth-of-type(17) > hanwooauct_0bamt').get_text())
    hanwoo_4b_amt.append(soup.select_one('items > item:nth-of-type(17) > hanwooauct_4bamt').get_text())
    pork_amt.append(soup.select_one('items > item:nth-of-type(17) > skinyauctamt').get_text())
    yukwoo_amt.append(soup.select_one('items > item:nth-of-type(17) > yukwooauctamt').get_text())
    
    
data_dict['도매시장명'] = stores
data_dict['경매일자'] = dates
data_dict['한우평균경매금액'] = hanwoo_amt
data_dict['한우1B경매단가'] = hanwoo_0b_amt
data_dict['한우3B경매단가'] = hanwoo_4b_amt
data_dict['돼지탕박경매금액'] = pork_amt
data_dict['육우경매금액'] = yukwoo_amt

df = pd.DataFrame(data_dict)

In [ ]:
df

## local mysql에 적재하는 코드 짜놓기

In [9]:
import pymysql
db = pymysql.connect(host='localhost',
                    port=3306,
                    user='younghun',
                    password='watson1259',
                    db='beef_pork_db',
                    charset='utf8')
cursor = db.cursor()

rows = map(list, zip(*data_dict.values()))

# 한 row씩 DB에 insert하기
for row in rows:
    sql = """INSERT INTO prices VALUES('"""+row[0]+"""','"""+row[1]+"""','"""+str(row[2])+"""',
    '"""+str(row[3])+"""','"""+str(row[4])+"""','"""+str(row[5])+"""','"""+str(row[6])+"""')"""
    cursor.execute(sql)
    db.commit()
    
db.close()